# Knock Language Model

Training language model from embeddings from the imbd training set

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import spacy

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
# Setting up the training and validation paths
TEST_PATH = 'test/'
TRAIN_PATH = 'train/'

In [3]:
%ls {TEST_PATH}

test_0.txt    test_190.txt  test_291.txt  test_3.txt    test_509.txt
test_103.txt  test_192.txt  test_292.txt  test_400.txt  test_516.txt
test_106.txt  test_198.txt  test_301.txt  test_411.txt  test_519.txt
test_113.txt  test_211.txt  test_306.txt  test_422.txt  test_520.txt
test_115.txt  test_213.txt  test_323.txt  test_426.txt  test_527.txt
test_11.txt   test_217.txt  test_326.txt  test_433.txt  test_52.txt
test_134.txt  test_22.txt   test_327.txt  test_434.txt  test_536.txt
test_137.txt  test_23.txt   test_331.txt  test_436.txt  test_541.txt
test_13.txt   test_243.txt  test_332.txt  test_439.txt  test_544.txt
test_145.txt  test_24.txt   test_334.txt  test_444.txt  test_554.txt
test_149.txt  test_252.txt  test_340.txt  test_448.txt  test_557.txt
test_152.txt  test_264.txt  test_346.txt  test_462.txt  test_56.txt
test_155.txt  test_26.txt   test_349.txt  test_465.txt  test_5.txt
test_158.txt  test_272.txt  test_350.txt  test_46.txt   test_60.txt
test_168.txt  test_275.txt  test_355.tx

In [4]:
FILES = dict(train=TRAIN_PATH, validation=TEST_PATH, test=TEST_PATH)
FILES

{'test': 'test/', 'train': 'train/', 'validation': 'test/'}

In [5]:
bs = 16
bptt = 20

In [6]:
spacy_tok = spacy.load('en')

In [7]:
TEXT = pickle.load(open(f'TEXT.pkl','rb'))

In [10]:
len(TEXT.vocab.itos)

1181

In [9]:
model_data = LanguageModelData.from_text_files("", TEXT, **FILES, bs=bs, bptt=bptt, min_freq=1)

In [9]:
model_data


In [11]:
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

In [12]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [13]:
learner = model_data.get_model(opt_fn, em_sz, nh, nl,
               dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [14]:
learner.load_encoder('imbd_rnn_model')

RuntimeError: While copying the parameter named encoder.weight, whose dimensions in the model are torch.Size([1181, 200]) and whose dimensions in the checkpoint are torch.Size([37392, 200]).